In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import linearmodels as lm
import statsmodels.api as sm
from linearmodels import PanelOLS
from linearmodels import RandomEffects
from linearmodels import PooledOLS
from linearmodels import FirstDifferenceOLS
from linearmodels import BetweenOLS
from linearmodels import FamaMacBeth
import sqlite3
from tqdm import tqdm
import dask.dataframe as dd
from dask import delayed, compute
from dask.diagnostics import ProgressBar

# Merge FM and HRCN to get smaller set (Only states with HRCN risk data)

In [28]:
db_path = "../Database/thesis_database.db"
conn = sqlite3.connect(db_path, check_same_thread=False)

In [3]:
hrcn_risk_agg = pd.read_sql("SELECT * FROM hrcn_risk_agg", conn)

In [4]:
# read'../../Data/mainland_usa_gdf_msa_aggregated.pkl'
mortgage_hrcn = {}
#Merge Mortgage data and Hurricane Data on MSA ||| This reduces the datasets by removing all unessential states data
pb = tqdm(range(1999, 2023))
for year in pb:
    pb.set_description(f"Merging {year} hrcn and mortgage data")
    query = f"""
    SELECT *
    FROM fm_{year}
    """
    df_year = pd.read_sql(query, conn) 
    merged_df = df_year.merge(hrcn_risk_agg, on='MSA', how='inner')
    # fm_combined = pd.concat([fm_combined, merged_df])
    key_name = f"fm_{year}"
    mortgage_hrcn[key_name] = merged_df
    del merged_df, df_year
#Runtime: 4:30

merging hrcn and mortgage data:   0%|          | 0/24 [00:00<?, ?it/s]

merging hrcn and mortgage data: 100%|██████████| 24/24 [04:21<00:00, 10.90s/it]


In [5]:
pb = tqdm(mortgage_hrcn.items())
for key, dataset in pb:
    pb.set_description(f"writing {key} to database")
    key_hrcn = f"{key}_hrcn"
    # Write the dataset to the database
    dataset.to_sql(key, conn, if_exists="replace", index=False)
    del dataset

writing fm_2022 to database: 100%|██████████| 24/24 [02:57<00:00,  7.38s/it]


# Combine Files

In [6]:
# Assuming you have already created a Dask SQL connection
# (if not, you'll need to set that up)
def fetch_and_merge(year):
    query = f"""
    SELECT *
    FROM fm_{year}_hrcn
    """
    merged_df = dd.from_pandas(pd.read_sql(query, conn), npartitions=10)  # Adjust npartitions based on your available cores and data size
    return merged_df

# Using list comprehension with Dask's delayed
results = [delayed(fetch_and_merge)(year) for year in range(1999, 2023)]

# Compute the results in parallel
with ProgressBar():
    merged_dataframes = compute(*results, scheduler='single-threaded')
# Ensure merged_dataframes is a list
merged_dataframes_list = list(merged_dataframes)
fm_combined = dd.concat(merged_dataframes_list)


[########################################] | 100% Completed | 237.19 s


In [7]:
fm_combined['FIRST_F'] = fm_combined['FIRST_F'].astype(str)
fm_combined.to_parquet('../Data/fm_combined.parquet', engine='pyarrow')

# Aggregate Dataset

In [7]:
fm_combined = dd.read_parquet('../Data/fm_combined.parquet')

In [8]:
# Step 1: Drop unnecessary columns
fm_combined = fm_combined.drop(columns=['ELTV', 'FPD', 'MD'], errors='ignore')
# Step 2: Create CLDS90 and CLDS180 columns
fm_combined['CLDS90'] = (fm_combined['CLDS'] == 3).astype(int)
fm_combined['CLDS180'] = (fm_combined['CLDS'] == 6).astype(int)
fm_combined['D90_month'] = 1
fm_combined['D180_month'] = 1


In [9]:
# Define your function as before
def create_indicators(group):
    d90_date = group[group['CLDS90'] == 1]['Date'].min()
    d180_date = group[group['CLDS180'] == 1]['Date'].min()
    if pd.notnull(d90_date):
        group['D90_month'] = (group['Date'] <= d90_date).astype(int)
    if pd.notnull(d180_date):
        group['D180_month'] = (group['Date'] <= d180_date).astype(int)
    return group

# Use Dask's groupby and apply methods
with ProgressBar():
    fm_combined = fm_combined.groupby('LSN').apply(create_indicators, meta=fm_combined).compute(scheduler='threads')
#22 minutes

[########################################] | 100% Completed | 21m 27s


In [35]:
#fm_combined to parquet
fm_combined.to_parquet('../Data/fm_combined_2.parquet', engine='pyarrow')

In [15]:
# Define a custom aggregation function
def custom_aggregation(group):
    agg_data = {
        'UNQ_LSN': group['LSN'].nunique(),
        'P_TYPE_MOST_FREQ': group['P_TYPE'].mode()[0],  # Using mode for most frequent in Pandas
        'D90': group[group['D90_month'] == 1]['CLDS90'].sum() / group[group['D90_month'] == 1]['LSN'].nunique(),
        'D180': group[group['D180_month'] == 1]['CLDS180'].sum() / group[group['D180_month'] == 1]['LSN'].nunique()
    }
    
    # Add mean for all other columns
    for col in group.columns:
        if col not in ['MSA', 'Date', 'LSN', 'P_TYPE', 'CLDS90', 'CLDS180', 'D90_month', 'D180_month']:
            try:
                agg_data[col] = group[col].mean()
            except:
                pass

    return pd.Series(agg_data)

# Wrap the groupby object with tqdm for progress bar
tqdm.pandas(desc="Aggregating Data")
aggregated = fm_combined.groupby(['MSA', 'Date']).progress_apply(custom_aggregation).reset_index()
aggregated['Date'] = pd.to_datetime(aggregated['Date'])
# Runtime: 

Aggregating Data: 100%|██████████| 77428/77428 [03:41<00:00, 349.91it/s]


In [36]:
del fm_combined

In [16]:
aggregated.head()

,MSA,Date,UNQ_LSN,P_TYPE_MOST_FREQ,D90,D180,CLDS,AGE,CIR,DDD,...,HRCN_EALA,HRCN_EALT,HRCN_EALS,HRCN_ALRB,HRCN_ALRP,HRCN_ALRA,HRCN_ALR_N,HRCN_RISKV,HRCN_RISKS,HRCN_EALS_Norm
0,10180,1999-03-01,1,SF,0.0,0.0,0.0,1.000000,6.625000,0.0,...,3450.990886,67062.792715,29.528751,0.000006,1.386015e-09,0.000087,31.546332,82619.358078,31.493943,-0.872395
1,10180,1999-04-01,1,SF,0.0,0.0,0.0,2.000000,6.625000,0.0,...,3450.990886,67062.792715,29.528751,0.000006,1.386015e-09,0.000087,31.546332,82619.358078,31.493943,-0.872395
2,10180,1999-05-01,1,SF,0.0,0.0,0.0,3.000000,6.625000,0.0,...,3450.990886,67062.792715,29.528751,0.000006,1.386015e-09,0.000087,31.546332,82619.358078,31.493943,-0.872395
3,10180,1999-06-01,2,SF,0.0,0.0,0.0,2.500000,7.062500,0.0,...,3450.990886,67062.792715,29.528751,0.000006,1.386015e-09,0.000087,31.546332,82619.358078,31.493943,-0.872395
4,10180,1999-07-01,3,SF,0.0,0.0,0.0,2.333333,7.291667,0.0,...,3450.990886,67062.792715,29.528751,0.000006,1.386015e-09,0.000087,31.546332,82619.358078,31.493943,-0.872395


In [6]:

# # Step 3 & 4: Group by and aggregate

# # Define a custom aggregation function
# def custom_aggregation(group):
#     agg_data = {
#         'UNQ_LSN': group['LSN'].nunique(),
#         'P_TYPE_MOST_FREQ': group['P_TYPE'].mode()[0],  # Using mode for most frequent in Pandas
#         'CLDS90': group[group['D90_month'] == 1]['CLDS90'].sum(),
#         'CLDS180': group[group['D180_month'] == 1]['CLDS90'].sum(),
#         'D90': group[group['D90_month'] == 1]['CLDS90'].sum() / group[group['D90_month'] == 1]['LSN'].nunique(),
#         'D180': group[group['D180_month'] == 1]['CLDS180'].sum() / group[group['D180_month'] == 1]['LSN'].nunique()
#     }
    
#     # Add mean for all other columns
#     for col in group.columns:
#         if col not in ['MSA', 'Date', 'LSN', 'P_TYPE', 'CLDS90', 'CLDS180']:
#             try:
#                 agg_data[col] = group[col].mean()
#             except:
#                 pass

#     return pd.Series(agg_data)

# tqdm.pandas()
# aggregated = fm_combined_df.groupby(['MSA', 'Date']).progress_apply(custom_aggregation).reset_index()
# aggregated['Date'] = pd.to_datetime(aggregated['Date'])

100%|██████████| 77428/77428 [01:45<00:00, 734.03it/s]


In [25]:
#save aggregated to parquet
aggregated.to_parquet('../Data/aggregated.parquet', engine='pyarrow')

In [44]:
aggregated = dd.read_parquet('../Data/aggregated.parquet')

In [45]:
# aggregated_df = aggregated.compute()
# aggregated_df['Date'] = pd.to_datetime(aggregated_df['Date'])

In [26]:
#loag hpi master
hpi_master = pd.read_excel('../Data/HPI_master.xls', dtype={'place_id': str, 'yr': int, 'period': int, 'index_nsa': float, 'quarter': str})
#keep place_id, yr, period and index_nsa
hpi_master = hpi_master[['place_id', 'yr', 'period', 'index_nsa', 'quarter']]
#add period column to aggregated for each quarter
aggregated['quarter'] = aggregated['Date'].dt.year.astype(str) + "Q" + aggregated['Date'].dt.quarter.astype(str)
#merge aggregated and hpi_master on place_id and period only keeping index_nsa
aggregated = aggregated.merge(hpi_master, left_on=['MSA','quarter'], right_on=['place_id', 'quarter'], how='left')

####Missing 2023q1 of 48060 and 13220. Moreover missing data from 19260 and 41780. Thats all!


### Load and merge MEI

In [29]:
#Get enso_mei
query = """
SELECT
    *
FROM enso_mei;
"""
enso_mei = pd.read_sql_query(query, conn)
enso_mei['Date'] = pd.to_datetime(enso_mei['Date'])
#merge enso_mei with aggregated_df
aggregated = aggregated.merge(enso_mei, on='Date', how='left')

### Load and merge HPI

In [30]:
#load UNRATE.csv by ; and merge with aggregated_df on Date
unrate = pd.read_csv('../Data/UNRATE.csv', sep=';')
#Convert Date to datetime format 01/01/1948
unrate['Date'] = pd.to_datetime(unrate['DATE'], format= '%d/%m/%Y')
#Only merge UNRATE column with aggregated_df
unrate = unrate[['Date', 'UNRATE']]
aggregated = aggregated.merge(unrate, how='left', on='Date')


In [56]:
#subset where index_nsa is null
# aggregated_df_null = aggregated[aggregated['index_nsa'].isnull()]


## Regression Models

In [47]:
fm_agg_model.columns

Index(['UNQ_LSN', 'P_TYPE_MOST_FREQ', 'D90', 'D180', 'CLDS', 'AGE', 'CIR',
       'DDD', 'CS', 'MIP', 'CLTV', 'DTI', 'LTV', 'OIR', 'OLT', 'HRCN_EVNTS',
       'HRCN_AFREQ', 'HRCN_EXP_A', 'HRCN_EXPB', 'HRCN_EXPP', 'HRCN_EXPPE',
       'HRCN_EXPA', 'HRCN_EXPT', 'HRCN_HLRB', 'HRCN_HLRP', 'HRCN_HLRA',
       'HRCN_EALB', 'HRCN_EALP', 'HRCN_EALPE', 'HRCN_EALA', 'HRCN_EALT',
       'HRCN_EALS', 'HRCN_ALRB', 'HRCN_ALRP', 'HRCN_ALRA', 'HRCN_ALR_N',
       'HRCN_RISKV', 'HRCN_RISKS', 'HRCN_EALS_Norm', 'quarter', 'place_id',
       'yr', 'period', 'index_nsa', 'Year', 'Month', 'MEI', 'Month_num',
       'UNRATE', 'DHRI', 'MEI2'],
      dtype='object')

In [32]:
from linearmodels.panel import PanelOLS, RandomEffects
# Convert the dataset into a panel structure
fm_agg_model = aggregated.set_index(['MSA', 'Date'])

# Define dependent variable and independent variables
dependent_var = fm_agg_model['D90']*100
fm_agg_model['DHRI'] = fm_agg_model['MEI'] * ((fm_agg_model['HRCN_EALS']- fm_agg_model['HRCN_EALS'].mean()) / fm_agg_model['HRCN_EALS'].std())
fm_agg_model['MEI2'] = fm_agg_model['MEI']**2

### Fixed Effects

In [49]:
# Run a fixed effects regression
exog_vars = ['MEI','LTV', 'UNRATE', 'index_nsa']
exog = fm_agg_model[exog_vars]
mod_fe = PanelOLS(dependent_var, exog, entity_effects=True)
fe_res = mod_fe.fit()
fe_res

/Users/yannickpichardo/.pyenv/versions/3.11.1/lib/python3.11/site-packages/linearmodels/panel/model.py:1214: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


Dep. Variable:,D90,R-squared:,0.0150
Estimator:,PanelOLS,R-squared (Between):,-0.2341
No. Observations:,99729,R-squared (Within):,0.0150
Date:,"Sun, Oct 29 2023",R-squared (Overall):,0.0066
Time:,19:10:55,Log-likelihood,-7.366e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,377.97
Entities:,268,P-value,0.0000
Avg Obs:,372.12,Distribution:,"F(4,99457)"
Min Obs:,222.00,,
Max Obs:,1160.0,F-statistic (robust):,377.97


In [50]:
# Run a fixed effects regression
exog_vars = ['DHRI', 'LTV', 'UNRATE', 'OIR', 'index_nsa']
exog = fm_agg_model[exog_vars]
mod_fe = PanelOLS(dependent_var, exog, entity_effects=True)
fe_res = mod_fe.fit()
fe_res

/Users/yannickpichardo/.pyenv/versions/3.11.1/lib/python3.11/site-packages/linearmodels/panel/model.py:1214: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


Dep. Variable:,D90,R-squared:,0.0154
Estimator:,PanelOLS,R-squared (Between):,-2.5202
No. Observations:,99729,R-squared (Within):,0.0154
Date:,"Sun, Oct 29 2023",R-squared (Overall):,-0.0920
Time:,19:11:01,Log-likelihood,-7.364e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,311.28
Entities:,268,P-value,0.0000
Avg Obs:,372.12,Distribution:,"F(5,99456)"
Min Obs:,222.00,,
Max Obs:,1160.0,F-statistic (robust):,311.28


In [51]:
# Run a fixed effects regression
exog_vars = ['HRCN_EALS', 'HRCN_AFREQ', 'LTV', 'UNRATE', 'OIR', 'index_nsa']
exog = fm_agg_model[exog_vars]
mod_fe = PanelOLS(dependent_var, exog, entity_effects=True)
fe_res = mod_fe.fit()
fe_res

/Users/yannickpichardo/.pyenv/versions/3.11.1/lib/python3.11/site-packages/linearmodels/panel/model.py:1214: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


AbsorbingEffectError: 
The model cannot be estimated. The included effects have fully absorbed
one or more of the variables. This occurs when one or more of the dependent
variable is perfectly explained using the effects included in the model.

The following variables or variable combinations have been fully absorbed
or have become perfectly collinear after effects are removed:

          HRCN_AFREQ, LTV, UNRATE, OIR
          HRCN_EALS

Set drop_absorbed=True to automatically drop absorbed variables.


### Random Effects

DHRI

In [53]:
# Run a random effects regression
exog_vars = ['MEI', 'MEI2','LTV', 'UNRATE', 'OIR', 'index_nsa']
exog = fm_agg_model[exog_vars]
mod_re = RandomEffects(dependent_var, exog)
re_res = mod_re.fit()
re_res

/Users/yannickpichardo/.pyenv/versions/3.11.1/lib/python3.11/site-packages/linearmodels/panel/model.py:2616: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


Dep. Variable:,D90,R-squared:,0.0293
Estimator:,RandomEffects,R-squared (Between):,0.8022
No. Observations:,99729,R-squared (Within):,0.0152
Date:,"Sun, Oct 29 2023",R-squared (Overall):,0.0502
Time:,19:11:36,Log-likelihood,-7.383e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,500.83
Entities:,268,P-value,0.0000
Avg Obs:,372.12,Distribution:,"F(6,99723)"
Min Obs:,222.00,,
Max Obs:,1160.0,F-statistic (robust):,500.83


In [54]:
# Run a random effects regression
exog_vars = ['DHRI', 'LTV', 'UNRATE', 'OIR', 'index_nsa']
exog = fm_agg_model[exog_vars]
mod_re = RandomEffects(dependent_var, exog)
re_res = mod_re.fit()
re_res

/Users/yannickpichardo/.pyenv/versions/3.11.1/lib/python3.11/site-packages/linearmodels/panel/model.py:2616: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


Dep. Variable:,D90,R-squared:,0.0298
Estimator:,RandomEffects,R-squared (Between):,0.8074
No. Observations:,99729,R-squared (Within):,0.0152
Date:,"Sun, Oct 29 2023",R-squared (Overall):,0.0505
Time:,19:11:50,Log-likelihood,-7.382e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,613.01
Entities:,268,P-value,0.0000
Avg Obs:,372.12,Distribution:,"F(5,99724)"
Min Obs:,222.00,,
Max Obs:,1160.0,F-statistic (robust):,613.01


HRCN

In [55]:
# Run a random effects regression
exog_vars = ['HRCN_EALS', 'HRCN_AFREQ', 'LTV', 'UNRATE', 'OIR', 'index_nsa']
exog = fm_agg_model[exog_vars]
mod_re = RandomEffects(dependent_var, exog)
re_res = mod_re.fit()
re_res

/Users/yannickpichardo/.pyenv/versions/3.11.1/lib/python3.11/site-packages/linearmodels/panel/model.py:2616: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


Dep. Variable:,D90,R-squared:,0.0308
Estimator:,RandomEffects,R-squared (Between):,0.8202
No. Observations:,99729,R-squared (Within):,0.0151
Date:,"Sun, Oct 29 2023",R-squared (Overall):,0.0509
Time:,19:11:56,Log-likelihood,-7.383e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,528.29
Entities:,268,P-value,0.0000
Avg Obs:,372.12,Distribution:,"F(6,99723)"
Min Obs:,222.00,,
Max Obs:,1160.0,F-statistic (robust):,528.29


### Mixed Model

In [56]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
aggregated_df_mm = aggregated.copy()
#drop rows with index_nsa nan
aggregated_df_mm = aggregated_df_mm.dropna(subset=['index_nsa'])
# Prepare the dataset
aggregated_df_mm['D90_scaled'] = aggregated_df_mm['D90'] * 100
aggregated_df_mm['DHRI'] = aggregated_df_mm['MEI'] * ((aggregated_df_mm['HRCN_EALS'] - aggregated_df_mm['HRCN_EALS'].mean()) / aggregated_df_mm['HRCN_EALS'].std())
# Create a formula for the mixed model with fixed effects for Date
formula = "D90_scaled ~ DHRI + LTV + UNRATE + OIR + C(Year) + index_nsa"
# Fit the mixed model with random intercepts for each 3ZIP
mixed_model = smf.mixedlm(formula, aggregated_df_mm, groups=aggregated_df_mm['MSA'])
mixed_result = mixed_model.fit()
print(mixed_result.summary())

          Mixed Linear Model Regression Results
Model:             MixedLM Dependent Variable: D90_scaled 
No. Observations:  99729   Method:             REML       
No. Groups:        268     Scale:              0.2557     
Min. group size:   222     Log-Likelihood:     -73752.9047
Max. group size:   1160    Converged:          Yes        
Mean group size:   372.1                                  
----------------------------------------------------------
                Coef.  Std.Err.   z    P>|z| [0.025 0.975]
----------------------------------------------------------
Intercept       -0.372    0.073 -5.124 0.000 -0.514 -0.230
C(Year)[T.2000] -0.022    0.013 -1.656 0.098 -0.048  0.004
C(Year)[T.2001]  0.010    0.013  0.782 0.434 -0.016  0.036
C(Year)[T.2002]  0.019    0.013  1.514 0.130 -0.006  0.044
C(Year)[T.2003]  0.055    0.013  4.332 0.000  0.030  0.079
C(Year)[T.2004]  0.067    0.014  4.838 0.000  0.040  0.094
C(Year)[T.2005]  0.104    0.015  6.865 0.000  0.074  0.134
C(Year)[

/Users/yannickpichardo/.pyenv/versions/3.11.1/lib/python3.11/site-packages/statsmodels/regression/mixed_linear_model.py:2238: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
